In [1]:
import json
import requests
import regex as re
from bs4 import BeautifulSoup

In [2]:
# Отримаємо дату
responce = requests.get('https://www.bbc.com/sport')
soup = BeautifulSoup(responce.text, features="lxml")

In [3]:
# Отримаємо посилання з основної сторінки
main_all_divs = soup.find_all("div")
main_divs = [div for div in main_all_divs if div.get("data-testid") == "promo" and div.get("type") == "article"]
main_links = ["https://www.bbc.com" + div.next.next.next.next.get("href") for div in main_divs if div.next.next.next.next.next.get("role") == "text"]
for i in main_links :
    print(i)
    print()

https://www.bbc.com/sport/football/articles/cjerlqzvx73o

https://www.bbc.com/sport/cricket/articles/c98qxp15y55o

https://www.bbc.com/sport/football/articles/cg3mwnd38nmo

https://www.bbc.com/sport/cycling/articles/cd1r3z1l4jjo

https://www.bbc.com/sport/tennis/articles/cqv53y38z07o

https://www.bbc.com/sport/golf/articles/c3g358pkr1eo

https://www.bbc.com/news/articles/ce78z0zg2ezo

https://www.bbc.com/sport/formula1/articles/cmm2z0l41p3o

https://www.bbc.com/sport/football/articles/ce4q7wngj4no

https://www.bbc.com/sport/football/articles/cley8lpy526o

https://www.bbc.com/sport/football/articles/c047djzzeezo

https://www.bbc.com/sport/football/articles/cx92vlq5z92o

https://www.bbc.com/sport/rugby-union/articles/cqql00291v2o

https://www.bbc.com/sport/football/articles/c4ngd96570zo

https://www.bbc.com/sport/rugby-union/articles/cd1jx817pwvo

https://www.bbc.com/sport/football/articles/cw5yvyq0yp9o

https://www.bbc.com/sport/football/articles/c0dm817xp2eo

https://www.bbc.com/sport/

In [4]:
# Беремо перші 5
links = main_links[:5]
responces = [requests.get(link).text for link in links]
soups = [BeautifulSoup(res, features="lxml") for res in responces]

In [5]:
# Беремо зв'язані теми
all_titles = []
all_links = []

print("---")
for i in range(0, len(links)):
    h2_raw = soups[i].find_all("h2")
    h2 = None
    for h2s in h2_raw :
        if h2s.text == "Related Topics" :
            h2 = h2s # Існує лише один підходящий h2
    if h2 == None :
        print("No related topics!!!")
        print("On link :", main_links[i])
        print("Number :", i + 1)
        print()
        print("---")
        print()
        continue
    related_topics_raw = h2.next.next.next

    related_topics_links = re.findall("href=\"([\w \/=+-]{1,})", str(related_topics_raw))
    related_topics_links = ["https://www.bbc.com" + link for link in related_topics_links]

    related_topics_titles = re.findall("([\w \"\'\/=+-]{1,})<\/a>", str(related_topics_raw))

    for j in range(0, len(related_topics_links)) :
        all_titles.append(related_topics_titles[j])
        all_links.append(related_topics_links[j])
        
        print(related_topics_titles[j])
        print(related_topics_links[j])
        print()
    print("---")
    print()

---
Football
https://www.bbc.com/sport/football

England Men's Football Team
https://www.bbc.com/sport/football/teams/england

---

England Men's Cricket Team
https://www.bbc.com/sport/cricket/teams/england

West Indies
https://www.bbc.com/sport/cricket/teams/west-indies

Cricket
https://www.bbc.com/sport/cricket

---

UEFA Euro 2024
https://www.bbc.com/sport/football/european-championship

Football
https://www.bbc.com/sport/football

---

Cycling
https://www.bbc.com/sport/cycling

---

Tennis
https://www.bbc.com/sport/tennis

---



In [7]:
# Зберігаємо
filename = "JSON.json"
data = [
    {"title": all_titles[i], "link" : all_links[i]}
    for i in range(0, len(all_links))
]
with open(filename, "w", encoding='utf-8') as f:
    json.dump(data, f, indent=2)